(sec:nto)=
# Natural transition orbitals

Natural transitions orbitals (NTOs) are designed to provide a compact representation of the transition density matrix {cite}`Martin2003`.

If we collect occupied and unoccupied (real) MOs as row  and column vectors, repsectively,

$$
\bar{\phi}_\mathrm{occ}(\mathbf{r})  = (\ldots, \phi_i(\mathbf{r}),  \ldots); \quad
\bar{\phi}_\mathrm{unocc}(\mathbf{r}) = (\ldots, \phi_a(\mathbf{r}),  \ldots)^T
$$

and scatter the (real) RPA eigenvector into a rectangular transition matrix $T$ of dimension $n_\mathrm{occ} \times n_\mathrm{unocc}$

$$
T_{f, ia} = Z_{f, ia} - Y_{f, ia}
$$

then we can write the transition moment as

\begin{equation*}
    \langle \Psi_f | \hat{\Omega} | \Psi_0 \rangle =
    X^T_f \Omega^{[1]} = 
    \big(
    Z_f^T, Y_f^T
    \big)
    \begin{pmatrix}
    g\\-g
    \end{pmatrix}
    =
    \big(
    Z_f - Y_f
    \big)^T
    g
    = 
    \int
    \omega(\mathbf{r}) \,
    \bar{\phi}_\mathrm{occ}(\mathbf{r}) \,
    T \,
    \bar{\phi}_\mathrm{unocc}(\mathbf{r}) \,
    d^3\mathbf{r}
\end{equation*}

where we have assumed the operator to be a scalar operator that can be moved in front of both orbitals in the integrand

$$
\hat{\Omega} = \sum_{i=1}^N \omega(\mathbf{r}_i)
$$

```{note}
In the Tamm–Dancoff approximation (TDA), the lower part of the excitation vector is left out from the calculation and we have $T_{f, ia} = Z_{f, ia}$.
```

By means of a singular value decomposition (SVD), we can factorize the transition matrix

$$
T = U \Lambda V
$$

where matrices $U$ and $V$ are unitary and $\Lambda$ is rectangular diagonal

$$
\Lambda =
\begin{bmatrix}
\sqrt{\lambda_1} & 0 & \cdots & 0 & 0 & \cdots & 0 \\
0 & \sqrt{\lambda_2} & \cdots & 0 & 0 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \sqrt{\lambda_\mathrm{nocc}} & 0 & \cdots & 0 \\
\end{bmatrix}
$$

Defining the diagonal elements as $\sqrt{\lambda_i}$ with a square root follows the original reference {cite}`Martin2003`.

The SVD transformation matrices define the pairs of hole and electron (or particle) NTOs

$$
\bar{\phi}_\mathrm{h}(\mathbf{r}) = \bar{\phi}_\mathrm{occ}(\mathbf{r}) \, U; \quad
\bar{\phi}_\mathrm{e}(\mathbf{r}) = V \, \bar{\phi}_\mathrm{unocc}(\mathbf{r})
$$

such that

\begin{equation*}
    \langle \Psi_f | \hat{\Omega} | \Psi_0 \rangle =
    \int
    \omega(\mathbf{r}) \,
    \bar{\phi}_\mathrm{h}(\mathbf{r}) \,
    \Lambda \,
    \bar{\phi}_\mathrm{e}(\mathbf{r}) \,
    d^3\mathbf{r}
\end{equation*}

Compared to the canonical MOs, the NTOs can provide an improved understanding of the nature of a given transition in cases when it involves multiple occupied and/or unoccupied MOs.

## Reference calculation

Let us illustrate the concept of NTOs with a study of the bright Frenkel excitonic state of the ethylene dimer. This is the $S_2$ state of the system as discussed in the section on [selection rules](sec:selection_rules) where also MO-plots are provided.

In [1]:
import numpy as np
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 6.


In [2]:
dimer_xyz = """12

C        0.67759997    0.00000000    -10.0
C       -0.67759997    0.00000000    -10.0
H        1.21655197    0.92414474    -10.0
H        1.21655197   -0.92414474    -10.0
H       -1.21655197   -0.92414474    -10.0
H       -1.21655197    0.92414474    -10.0
C        0.67759997    0.00000000     10.0
C       -0.67759997    0.00000000     10.0
H        1.21655197    0.92414474     10.0
H        1.21655197   -0.92414474     10.0
H       -1.21655197   -0.92414474     10.0
H       -1.21655197    0.92414474     10.0"""

In [3]:
molecule = vlx.Molecule.read_xyz_string(dimer_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31g", ostream=None)

We employ the BHANDHLYP functional to get a physically reasonable description of the $\pi\pi^*$-excited states.

In [4]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_drv.xcfun = "bhandhlyp"

scf_results = scf_drv.compute(molecule, basis)

We request NTOs to be calculated.

In [5]:
lreig_drv = vlx.LinearResponseEigenSolver()
lreig_drv.ostream.mute()

lreig_drv.nstates = 2
lreig_drv.nto = True
lreig_drv.nto_cubes = False

lreig_results = lreig_drv.compute(molecule, basis, scf_results)

The lambda values associated with the NTOs are returned from the `compute()` method. We print those for state $S_2$. 

In [39]:
state = 1 # Python indexing of excited states [0, 1, ...]

In [37]:
print(lreig_results["nto_lambdas"][state])

[4.94582065e-01 4.94551408e-01 1.02208591e-02 1.02198930e-02
 3.22403391e-03 3.22274858e-03 2.46371242e-03 2.46336316e-03
 1.75911027e-03 1.75888592e-03 8.95327254e-04 8.94745051e-04
 1.26847719e-07 1.26578358e-07 9.39753079e-08 9.36320624e-08]


## Visualization

The NTOs can be viewed with the `OrbitalViewer` class by supplying the name of the file in which the orbitals are stored. There is one file per transition and the filenames are returned from the `compute()` method. 

```{note}
The order of the hole orbitals is reversed such that hole orbital with the largest $\lambda$-value is found as number $n_\mathrm{occ}$ in the orbital viewer. The associated electron orbital is found as number $n_\mathrm{occ} + 1$. And so forth.
```

In [16]:
viewer = vlx.OrbitalViewer()
viewer.plot(molecule, basis, lreig_results["nto_h5_files"][state])

Output()

Dropdown(description='Orbital:', index=15, options=(('  1 occ=-0.000 ene=0.000', 0), ('  2 occ=-0.000 ene=0.00…

## Implementation

Below follows an implementation of the NTO concept. First, we find the dimensions of the involved vectors and matrices.

In [17]:
norb = basis.get_dimension_of_basis()
nocc = molecule.number_of_alpha_electrons()
nvirt = norb - nocc
nexc = nocc * nvirt

print("Number of orbitals:", norb)
print("Number of occupied orbitals:", nocc)
print("Number of unoccupied orbitals:", nvirt)
print("Number of excitations:", nexc)

Number of orbitals: 52
Number of occupied orbitals: 16
Number of unoccupied orbitals: 36
Number of excitations: 576


The RPA eigenvector is obtained with use of the `get_full_solution_vector()` method and thereafter scattered into the transition matrix that becomes SVD factorized. 

In [41]:
Xf = lreig_drv.get_full_solution_vector(
    lreig_results["eigenvectors_distributed"][state]
)

Zf = Xf[:nexc]
Yf = Xf[nexc:]
T = np.reshape(Zf - Yf, (nocc, nvirt))
U, L, V = np.linalg.svd(T)

print("Square of diagonal elements:\n", L**2)

Square of diagonal elements:
 [3.72499014e-01 3.72475472e-01 2.29239846e-02 2.29211293e-02
 6.24552563e-03 6.24323135e-03 4.88180538e-03 4.88178175e-03
 3.99380771e-03 3.99361417e-03 2.04378291e-03 2.04255914e-03
 4.28944806e-07 4.28167413e-07 3.18819613e-07 3.17971237e-07]


We note that these $\lambda$-values are in perfect agreement with those obtained in the reference calculation.